In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import itertools
import json
from pathlib import Path
from datetime import timedelta, datetime
from scipy.sparse import csr_matrix, vstack


import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [7]:
# Path("/media/hpi_share/")#Path("//FS23/projekte$/MP2021/MPWS2021/MPWS2021FN1")
mp_drive_dir = Path("../../../")
mp_plot_dir = mp_drive_dir / "plots"
input_path = Path("../../matched-infoboxes-extracted/")
input_data = list(input_path.rglob("*.json"))
files = [x for x in input_data if x.is_file()]
len(files)  # total 580

70

In [8]:
num_iteratons = 1
number_of_files = 10
number_of_files_start = 5
num_edits = 0
num_change_tuples = 0
typo_lst = []
for _ in range(num_iteratons):
    revision_tuples = []
    change_tuples = []
    for file in tqdm(files[number_of_files_start:number_of_files_start+number_of_files]):
        with open(file, 'r', encoding='utf-8') as f:
            for jsonObj in f:
                single_edit = json.loads(jsonObj)
                num_edits += 1
                title = single_edit['pageTitle']
                pageID = single_edit['pageID']
                key = single_edit['key']
                template = single_edit['template'] if 'template' in single_edit.keys(
                ) else None
                changes = single_edit['changes']
                validFrom = single_edit['validFrom']
                validTo = single_edit['validTo'] if 'validTo' in single_edit.keys(
                ) else None
                revisionId = single_edit['revisionId']
                changeType = single_edit['type']
                attributes = single_edit['attributes'] if 'attributes' in single_edit.keys(
                ) else None
                user_name = single_edit['user']['username'] if 'username' in single_edit['user'].keys(
                ) else None
                user_id = single_edit['user']['id'] if 'id' in single_edit['user'].keys(
                ) else None
                user_ip = single_edit['user']['ip'] if 'ip' in single_edit['user'].keys(
                ) else None
                revision_tuples.append(
                    (title, pageID, key, template, revisionId, changeType, validFrom, validTo, user_name, user_id, user_ip, attributes))

                for change in changes:
                    num_change_tuples += 1
                    name = change['property']['name']
                    current_value = change['currentValue'] if 'currentValue' in change.keys(
                    ) else None
                    previous_value = change['previousValue'] if 'previousValue' in change.keys(
                    ) else None
                    validTo = change['valueValidTo'] if 'valueValidTo' in change.keys(
                    ) else None
                    change_tuples.append((title, pageID, key, template, name, previous_value,
                                          current_value, validFrom, validTo, revisionId, changeType, user_name, user_id, user_ip, attributes))
    number_of_files_start += number_of_files


  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
# data = pd.DataFrame(change_tuples, columns=['key', 'pageID', 'title', 'user','name', 'previous_value', 'current_value', 'timestamp', 'edit_type'])
# data['timestamp'] = pd.to_datetime(data['timestamp']).dt.tz_localize(None)
data = pd.DataFrame(change_tuples, columns=['pageTitle', 'pageID', 'key', 'template', 'name', 'previous_value',
                    'current_value', 'validFrom', 'validTo', 'revisionId', 'change_type', 'user_name', 'user_id', 'user_ip', 'attributes'])
data['validFrom'] = pd.to_datetime(data['validFrom'])
data['validTo'] = pd.to_datetime(data['validTo'])


In [10]:
data.head(3)

,pageTitle,pageID,key,template,name,previous_value,current_value,validFrom,validTo,revisionId,change_type,user_name,user_id,user_ip,attributes
0,Indonesian National Armed Forces,14650,119306865-0,infobox national military,reaching,None,"2,201,047",2007-03-31 18:02:53+00:00,2008-07-25 09:13:54+00:00,119306865,CREATE,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
1,Indonesian National Armed Forces,14650,119306865-0,infobox national military,image,None,{{border|[[Image:Flag of Indonesia.svg|110px]]}},2007-03-31 18:02:53+00:00,2007-06-21 05:23:24+00:00,119306865,CREATE,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
2,Indonesian National Armed Forces,14650,119306865-0,infobox national military,available,None,"60,543,028",2007-03-31 18:02:53+00:00,2010-02-25 13:42:35+00:00,119306865,CREATE,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."


In [11]:
print("Unique user ids:", data['user_id'].nunique())
print("Unique user ips:", data['user_ip'].nunique())
print("Unique users (unique ip/id combinations):", len(data[['user_id', 'user_ip']].drop_duplicates())) #including both NaN/None
#data['user_id'].nunique()+data['user_ip'].nunique()

Unique user ids: 52238
Unique user ips: 106723
Unique users (unique ip/id combinations): 158962


In [12]:
# user ids and ips are none?
data.query('user_id.isnull() and user_ip.isnull()').head(3)

,pageTitle,pageID,key,template,name,previous_value,current_value,validFrom,validTo,revisionId,change_type,user_name,user_id,user_ip,attributes
23438,Erectile dysfunction,14783,30023178-0,None,name,Impotence of organic origin,None,2007-05-03 01:21:00+00:00,2007-05-03 02:01:23+00:00,127839960,DELETE,None,NaN,None,None
23439,Erectile dysfunction,14783,30023178-0,None,icd10,N48.4,None,2007-05-03 01:21:00+00:00,2007-05-03 02:01:23+00:00,127839960,DELETE,None,NaN,None,None
23440,Erectile dysfunction,14783,30023178-0,None,icd9,{{ICD9|607.84}},None,2007-05-03 01:21:00+00:00,2007-05-03 02:01:23+00:00,127839960,DELETE,None,NaN,None,None


In [13]:
# Mostly Deletes and Update have no user
data.query('user_id.isnull() and user_ip.isnull()').groupby('change_type').count()

,pageTitle,pageID,key,template,name,previous_value,current_value,validFrom,validTo,revisionId,user_name,user_id,user_ip,attributes
change_type,,,,,,,,,,,,,,
DELETE,2413,2413,2413,0,2413,2413,0,2413,2413,2413,0,0,0,0
UPDATE,6,6,6,6,6,6,6,6,6,6,0,0,0,6


In [14]:
# sanity check if one revid has one user
data[["revisionId","user_id","user_ip"]].groupby("revisionId").nunique().head(3)

,user_id,user_ip
revisionId,,
4710414,1,0
4744970,1,0
4864058,1,0


In [15]:
data[["revisionId","user_id","user_ip"]].groupby(["user_id","user_ip"])

In [23]:
data_revs = pd.DataFrame(revision_tuples, columns=['pageTitle', 'pageID', 'key', 'template', 'revisionId', 'change_type', 'validFrom', 'validTo', 'user_name', 'user_id', 'user_ip', 'attributes'])
data_revs['validFrom'] = pd.to_datetime(data_revs['validFrom'])
data_revs['validTo'] = pd.to_datetime(data_revs['validTo'])
data_revs.head(3)

,pageTitle,pageID,key,template,revisionId,change_type,validFrom,validTo,user_name,user_id,user_ip,attributes
0,Indonesian National Armed Forces,14650,119306865-0,infobox national military,119306865,CREATE,2007-03-31 18:02:53+00:00,2007-03-31 18:03:27+00:00,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
1,Indonesian National Armed Forces,14650,119306865-0,infobox national military,119306983,UPDATE,2007-03-31 18:03:27+00:00,2007-06-21 05:23:24+00:00,Octane,986956.0,None,"{'country': 'Indonesia', 'domestic_suppliers':..."
2,Indonesian National Armed Forces,14650,119306865-0,infobox national military,139595000,UPDATE,2007-06-21 05:23:24+00:00,2007-08-30 21:00:35+00:00,Andrwsc,898126.0,None,"{'country': 'Indonesia', 'domestic_suppliers':..."


In [17]:
print("max creations/updates/deletions of a single user")
print("Create",data_revs.query('change_type=="CREATE"').groupby("user_id").count()['revisionId'].max())
print("Delete",data_revs.query('change_type=="DELETE"').groupby("user_id").count()['revisionId'].max())
print("Update",data_revs.query('change_type=="UPDATE"').groupby("user_id").count()['revisionId'].max())

max creations/updates/deletions of a single user
Create 3513
Delete 117
Update 5313


In [22]:
print("pages changed by userID")
print(data_revs.groupby("user_id").count()["pageID"].describe())
print("")
print("pages changed by userIP")
print(data_revs.groupby("user_ip").count()["pageID"].describe())

pages changed by userID
count    52238.000000
mean         5.298920
std         44.583251
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       7796.000000
Name: pageID, dtype: float64

pages changed by userIP
count    106723.000000
mean          1.509019
std           1.552623
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         111.000000
Name: pageID, dtype: float64


In [56]:
print("name changed by userID")
print(data.groupby("user_id").count()["name"].describe())

name changed by userID
count    52238.000000
mean        27.856082
std        583.071709
min          1.000000
25%          1.000000
50%          3.000000
75%         13.000000
max      99708.000000
Name: name, dtype: float64


In [19]:
print("pages and the number of users workd on")
data_revs.groupby("pageID").count()["user_id"].describe()

pages and the number of users workd on


count    2060.000000
mean      134.371359
std       229.117509
min         0.000000
25%        16.000000
50%        49.000000
75%       139.000000
max      2297.000000
Name: user_id, dtype: float64

In [52]:
data_revs["validFrom_time"]=data_revs["validFrom"].apply(lambda x: x.time())
data_revs["validFrom_hour"]=data_revs["validFrom"].apply(lambda x: x.time().hour)

In [55]:
data_revs.groupby("user_id").validFrom_hour.aggregate(
    ['min', 'max', 'median','mean', 'std'])  # .apply(lambda x:x["validFrom"])

,min,max,median,mean,std
user_id,,,,,
1.0,1,1,1.0,1.000000,NaN
2.0,0,18,14.0,9.000000,7.053368
4.0,0,21,15.0,12.800000,7.758866
11.0,8,21,20.0,16.333333,7.234178
24.0,9,21,15.0,15.000000,8.485281
...,...,...,...,...,...
37245318.0,22,22,22.0,22.000000,NaN
37246080.0,3,3,3.0,3.000000,0.000000
37251209.0,20,21,21.0,20.750000,0.500000


In [59]:
data_revs.groupby("user_id").validFrom_hour.aggregate(
    ['min', 'max', 'median','mean', 'std']).median()

min       11.000000
max       16.000000
median    14.000000
mean      13.333333
std        3.384044
dtype: float64

In [75]:
df=data.groupby(['key','name','user_id'])['pageID'].count().sort_values(ascending=False)
df[df>5]

key          name              user_id   
199558619-0  branches          20319104.0    134
63904096-0   label             3436807.0     122
48610185-1   genre             1112118.0      96
9271881-0    footnote_b        19683425.0     93
100170704-0  pop               6101321.0      78
                                            ... 
187480678-2  caption           6079914.0       6
38572925-0   nation            1879647.0       6
28104509-0   work_institution  1574574.0       6
33392431-0   agency            13286072.0      6
68851495-0   birth_date        409043.0        6
Name: pageID, Length: 18880, dtype: int64

In [90]:
data.query("key=='199558619-0' and name =='branches' and user_id=='20319104.0'")

,pageTitle,pageID,key,template,name,previous_value,current_value,validFrom,validTo,revisionId,change_type,user_name,user_id,user_ip,attributes
726326,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,{{flagicon image|Kuwait Army Flag 1978.png}} [...,{{flagicon image|Kuwait Army Flag 1978.png}} [...,2014-01-03 22:16:22+00:00,2014-01-03 22:18:56+00:00,589044451,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."
726327,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,{{flagicon image|Kuwait Army Flag 1978.png}} [...,{{flagicon image|Kuwait Army Flag 1978.png}} [...,2014-01-03 22:18:56+00:00,2014-01-03 22:25:04+00:00,589044828,UPDATE,OJOM,20319104.0,None,"{'country': 'State of Kuwait', 'domestic_suppl..."
726330,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,{{flagicon image|Kuwait Army Flag 1978.png}} [...,[[Kuwaiti Army|Kuwaiti Army]]<br />[[Kuwaiti N...,2014-01-03 22:25:04+00:00,2014-01-03 22:32:58+00:00,589045623,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."
726331,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,[[Kuwaiti Army|Kuwaiti Army]]<br />[[Kuwaiti N...,[[Kuwaiti Army|Kuwaiti Army]]<br />[[Kuwaiti N...,2014-01-03 22:32:58+00:00,2014-01-04 06:07:33+00:00,589046646,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."
726355,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,[[Kuwaiti Army]]<br />[[Kuwaiti Navy|Kuwaiti N...,[[Kuwaiti Army]]<br />[[Kuwaiti Navy|Kuwaiti N...,2014-02-26 10:31:08+00:00,2014-02-26 10:31:53+00:00,597205327,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726619,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,* Defense Security Force [[Cavalry]] & [[Infan...,* Defense Security Force [[Cavalry]] & [[Infan...,2016-03-03 21:54:31+00:00,2016-04-06 10:06:45+00:00,708140916,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."
726628,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,* Defense Security Force [[Cavalry]] & [[Infan...,* Defense Security Force [[Cavalry]] & [[Infan...,2016-04-06 10:06:45+00:00,2016-04-17 08:59:05+00:00,713880253,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."
726630,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,* Defense Security Force [[Cavalry]] & [[Infan...,* Kuwait Army [[Cavalry]] & [[Infantry]]\n(191...,2016-04-17 08:59:05+00:00,2016-04-22 21:11:58+00:00,715676024,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."
726633,Kuwait Military Forces,16692,199558619-0,infobox national military,branches,* Kuwait Army [[Cavalry]] & [[Infantry]]\n(191...,* [[Flag of Kuwait|Kuwait]] Army [[Cavalry]] &...,2016-04-22 21:11:58+00:00,2016-04-24 07:06:49+00:00,716631589,UPDATE,OJOM,20319104.0,None,"{'country': 'Kuwait', 'domestic_suppliers': ''..."


- How many different pages a user works on
- how many different users work on a single page
- at which time a user makes changes
- does a user make changes in a specific order multiple times
- user property correlation